# Ceny tepla 2017


Energetický regulační úřad pravidelně zveřejňuje [Přehled cen tepelné energie v členění podle cenových lokalit](https://www.eru.cz/teplo/statistika/prehled-cen-tepelne-energie-v-cleneni-podle-cenovych-lokalit) ve formátu pdf.

[Výsledné ceny tepla za rok 2017](http://www.eru.cz/documents/10540/462926/Vysledne_ceny_tepla_2017.pdf)


In [1]:
import camelot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Get pdf from the web and use Camelot to retrieve tables
tables = camelot.read_pdf('http://www.eru.cz/documents/10540/462926/Vysledne_ceny_tepla_2017.pdf', pages='1-end')
tables

<TableList n=41>

In [2]:
# Prepare header and labels
header = ['lokalita', 'kraj', 'pod_uhli', 'pod_plyn', 'pod_bio', 'pod_olej',\
          'pod_ost', 'vykon', 'pocet_odberat', 'pocet_mist', 'nad_10_czk', 'nad_10_gj',\
          'pod_10_czk','pod_10_gj','cptv_czk', 'cptv_gj',\
          'prim_rozv_czk', 'prim_rozv_gj','cvs_czk', 'cvs_gj',\
          'cvs_voda_czk', 'cvs_voda_gj','blok_kot_czk', 'blok_kot_gj',\
          'sek_rozv_czk', 'sek_rozv_gj',\
          'dps_czk', 'dps_gj', 'dom_kot_czk', 'dom_kot_gj']
podily = ['pod_uhli', 'pod_plyn', 'pod_bio', 'pod_olej', 'pod_ost']
labels = ['uhlí', 'zemní plyn', 'biomasa', 'topný olej', 'jiné']

In [3]:
# Merge all tables into one DataFrame
ceny = []
for i in range(tables.n):
    ceny.extend(tables[i].data[2:])
ceny = pd.DataFrame(ceny, columns=header)

In [4]:
# Clean strings (decimal, thousand), ignore values split on two rows (\n) and convert to numeric types
ceny.loc[:, 'pod_uhli':] = ceny.loc[:, 'pod_uhli':].apply(lambda x: x.str.replace(',', '.'))
ceny.loc[:, 'pod_uhli':] = ceny.loc[:, 'pod_uhli':].apply(lambda x: x.str.replace(' ', ''))
ceny.loc[:, 'pod_uhli':] = ceny.loc[:, 'pod_uhli':].apply(pd.to_numeric, errors='coerce')
ceny = ceny.fillna(0)
ceny.loc[:, 'pocet_odberat':'pocet_mist'] = ceny.loc[:, 'pocet_odberat':'pocet_mist'].astype('int64')
ceny.loc[:, 'pod_uhli':'pod_ost'] =  ceny.loc[:, 'pod_uhli':'pod_ost']/100

In [5]:
# Agregate energy and price from various supply options
gj = ceny.filter(regex='_gj+', axis=1).columns
czk = ceny.filter(regex='_czk+', axis=1).columns
ceny['dod_cena'] = np.average(ceny[czk], weights=ceny[gj], axis=1)
ceny['dod_mnozstvi'] = np.sum(ceny[gj], axis=1)

ceny.dod_mnozstvi.sum()
ceny.dod_mnozstvi.sum() == ceny.filter(regex='_gj+', axis=1).sum().sum()

True

In [6]:
ceny.to_csv('eru_ceny_tepla_2017.csv')
ceny

,lokalita,kraj,pod_uhli,pod_plyn,pod_bio,pod_olej,pod_ost,vykon,pocet_odberat,pocet_mist,...,blok_kot_czk,blok_kot_gj,sek_rozv_czk,sek_rozv_gj,dps_czk,dps_gj,dom_kot_czk,dom_kot_gj,dod_cena,dod_mnozstvi
0,Abertamy,K,0.0,1.0,0.0,0.0,0.0,0.810,3,2,...,575.22,1311.0,0.0,0.0,0.00,0.0,0.00,0.0,575.22,1979.0
1,Adamov,B,0.0,1.0,0.0,0.0,0.0,2.254,1,1,...,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,254.62,21824.0
2,Adamov - Opletalova 38 a 22,B,0.0,1.0,0.0,0.0,0.0,0.460,2,2,...,0.00,0.0,0.0,0.0,0.00,0.0,609.66,1520.0,609.66,1520.0
3,Adamov - P. Jilemnického 18 (K 72),B,0.0,1.0,0.0,0.0,0.0,0.090,1,1,...,0.00,0.0,0.0,0.0,0.00,0.0,325.22,547.0,325.22,547.0
4,Adamov - teplovodní zdroj,B,0.0,1.0,0.0,0.0,0.0,9.000,37,28,...,0.00,0.0,0.0,0.0,546.06,28987.0,0.00,0.0,546.06,28987.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2993,Žernůvka,B,0.0,1.0,0.0,0.0,0.0,0.300,1,2,...,0.00,0.0,0.0,0.0,0.00,0.0,456.84,1307.0,456.84,1307.0
2994,Židlochovice,B,0.0,1.0,0.0,0.0,0.0,0.863,15,30,...,0.00,0.0,0.0,0.0,0.00,0.0,438.33,3521.0,438.33,3521.0
2995,Žihle - Nový Dvůr 2,P,0.0,0.0,0.0,1.0,0.0,0.063,1,1,...,0.00,0.0,0.0,0.0,0.00,0.0,826.23,24.0,826.23,24.0
2996,Žinkovy,P,0.0,1.0,0.0,0.0,0.0,0.490,1,1,...,0.00,0.0,0.0,0.0,0.00,0.0,419.46,3071.0,419.46,3071.0
